# Model Training

Preprocess Data, build SGD model for E AND I only, compare scores between unbalanced and balanced dataset

In [1]:
##### IMPORTS

import pandas as pd


from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import classification_report, balanced_accuracy_score

In [2]:
INPUT_ROWS = 1000 # MAX 20000
BIG_DATA_ROWS = 400 # Less than 7500

TRAINING_MBTI_TYPE = "e_i"

col_list = ['e_i',"s_n",'f_t',"p_j", 'type', 'text']


if INPUT_ROWS < BIG_DATA_ROWS or BIG_DATA_ROWS > 7500 or INPUT_ROWS > 20000:
    ValueError("ArithmeticError detected")

In [3]:
from data.data import Data

# PreProcess it ALL!!!!
## SIKE NOT ENOUGH MEMORY
# LIMIT TO 10K ROWS INPUT #

data_raw = Data().get_all_data()

n_1 = int((INPUT_ROWS - BIG_DATA_ROWS) / 2)


# 8670 ROWS INPUT #
df1 = data_raw['twitter_MBTI']
df1 = df1[['label', 'text']]
df1.rename(columns={'label': 'type'}, inplace=True)
df1 = df1.sample(n = n_1, ignore_index = True, random_state = 1)

# 106062 ROWS INPUT #
df2 = data_raw['MBTI 500']
df2 = df2[['type', 'posts']]
df2.rename(columns={'posts': 'text'}, inplace=True)
df2 = df2.sample(n = BIG_DATA_ROWS, ignore_index = True, random_state = 1) # > 15000 combined, > 20000 df1 + df2

## SPECIFCALLY CREATING DIV 0 ERROR, ASK ABOUT IT ##
# 7811 ROWS INPUT #
df3 = data_raw['mbti_1']
df3.rename(columns={'posts': 'text'}, inplace=True)
df3 = df3.sample(n = n_1, ignore_index = True, random_state = 1)
# ####################################################

## Combined all data for PP if desired
data_combined = pd.concat([df3], ignore_index=True)
data_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    300 non-null    object
 1   text    300 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


/tmp/ipykernel_19891/1106222648.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'label': 'type'}, inplace=True)


In [4]:
from scripts.Preprocessing_full import training_preprocessing

# Preprocessing Pipeline on Training Data

df_pp = training_preprocessing(data_combined)

final dataset contains 300 rows and 2945 columns


In [5]:
df1.columns.to_list()[1][1]

'e'

In [6]:
# # Drop unnecessary columns and lower all strings
for col in col_list:
    if col == TRAINING_MBTI_TYPE:
        col_list.remove(col)

df_pp[TRAINING_MBTI_TYPE] = df_pp[TRAINING_MBTI_TYPE].str.lower()
df_pp_ready = df_pp.drop(columns=col_list)

In [7]:
def sampling_n_elements(group, n):
    if len(group) < n:
        return group
    return group.sample(n)

def balancer(df, col):
    col_1 = col[0]
    col_2 = col[2]
    
    val_counts_dict = df[col].value_counts().to_dict()
    
    if val_counts_dict[col_1] < val_counts_dict[col_2]:
        n = val_counts_dict[col_1]
    else:
        n = val_counts_dict[col_2]

    balanced_df = df.groupby(col).apply(lambda x: sampling_n_elements(x, n = n)).reset_index(drop=True)
    
    return balanced_df

In [8]:
## Unbalanced Dataset (No Change)
df_unbalanced = df_pp_ready

## Balanced Dataset, Limit rows to the same

df_balanced = balancer(df_pp_ready, TRAINING_MBTI_TYPE)

In [9]:
print(f"Unbalanced DF {TRAINING_MBTI_TYPE}: \n{df_unbalanced[TRAINING_MBTI_TYPE].value_counts().to_dict()}")
print(f"Balanced DF {TRAINING_MBTI_TYPE}: \n{df_balanced[TRAINING_MBTI_TYPE].value_counts().to_dict()}")

Unbalanced DF e_i: 
{'i': 245, 'e': 55}
Balanced DF e_i: 
{'e': 55, 'i': 55}


In [10]:
def PredictDict(model, X_test, y_test):
    y_pred = cross_val_predict(model, X_test, y_test, cv = 10)
    report = classification_report(y_test, y_pred, output_dict=True)
    bal_acc = balanced_accuracy_score(y_pred, y_test)
    report['bal_acc'] = bal_acc
    return report

In [11]:
# MODELS #
from sklearn.linear_model import SGDClassifier, LogisticRegression


model = SGDClassifier()

In [12]:
## Create Function to Instantiate Model, fit model, and evaluate

def Model_Tester(dataset, 
                 model_selection,
                 random_state = 1,
                 Prediction = True):
    """
        Test a machine learning model on a given dataset.

    Args:
        dataset (pandas.DataFrame): The dataset to be used for testing the model.
            The first element of the dataset is the target variable (`y`), 
            and everything else is the feature variable (`X`).
        model_selection: The machine learning model to be tested.
        random_state (int): Random seed for reproducibility. Default is 1.
        Prediction (bool): Indicates whether to perform prediction and return prediction results.
            Default is True.

    Returns:
        dict or model: If `Prediction` is True, a dictionary containing the prediction results is returned.
            The keys are 'y_true' (true target values), 'y_pred' (predicted target values),
            and 'accuracy' (accuracy score of the model). 
            If `Prediction` is False, the trained model is returned.
    """
    
    
    # Whatever y and X are, change here
    y = dataset.iloc[:,[0]]
    X = dataset.drop(columns = dataset.columns[0])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.3,
                                                    random_state=random_state) 
    
    
    model = model_selection
    
    model.fit(X_train, y_train)
    
    if Prediction == True:
        prediction_dict = PredictDict(model, X_test, y_test)
        return prediction_dict
    
    return model

In [13]:
history_unbalanced = Model_Tester(df_unbalanced,
                                     model)

/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/

In [14]:
history_balanced = Model_Tester(df_balanced,
                                     model)

/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/

In [16]:
history_balanced

{'e': {'precision': 0.47368421052631576,
  'recall': 0.75,
  'f1-score': 0.5806451612903226,
  'support': 12},
 'i': {'precision': 0.7857142857142857,
  'recall': 0.5238095238095238,
  'f1-score': 0.6285714285714286,
  'support': 21},
 'accuracy': 0.6060606060606061,
 'macro avg': {'precision': 0.6296992481203008,
  'recall': 0.6369047619047619,
  'f1-score': 0.6046082949308755,
  'support': 33},
 'weighted avg': {'precision': 0.6722488038277511,
  'recall': 0.6060606060606061,
  'f1-score': 0.6111436950146628,
  'support': 33},
 'bal_acc': 0.6296992481203008}

In [15]:
print(f"{INPUT_ROWS} Rows, Unbalanced {TRAINING_MBTI_TYPE}: F1 = {100*round(history_unbalanced['macro avg']['f1-score'],5)}%")
print("\n")
print(f"{INPUT_ROWS} Rows, Balanced {TRAINING_MBTI_TYPE}: F1 = {100*round(history_balanced['macro avg']['f1-score'],5)}%")


1000 Rows, Unbalanced e_i: F1 = 53.528%


1000 Rows, Balanced e_i: F1 = 60.461%


### DF1

3000 Rows, Unbalanced e_i: F1 = 52.01%

3000 Rows, Balanced e_i: F1 = 50.566%

10000 Rows, Unbalanced e_i: F1 = 52.101%

10000 Rows, Balanced e_i: F1 = 53.561%

### DF2

3000 Rows, Unbalanced e_i: F1 = 56.65%


3000 Rows, Balanced e_i: F1 = 61.818%


10000 Rows, Unbalanced e_i: F1 = 64.633%


10000 Rows, Balanced e_i: F1 = 63.597%


### DF3
3000 Rows, Unbalanced e_i: F1 = 57.831999999999994%


3000 Rows, Balanced e_i: F1 = 55.196%

### 1000 Rows, SGD

1000 Rows, Unbalanced e_i: F1 = 57.65%

1000 Rows, Balanced e_i: F1 = 52.94%


1000 Rows, Unbalanced s_n: F1 = 50.243%


1000 Rows, Balanced s_n: F1 = 54.301%

1000 Rows, Unbalanced f_t: F1 = 67.271%


1000 Rows, Balanced f_t: F1 = 65.001%

1000 Rows, Unbalanced p_j: F1 = 54.348%


1000 Rows, Balanced p_j: F1 = 53.416%

### 10000 Rows, SGD

10000 Rows, Unbalanced e_i: F1 = 56.045%


10000 Rows, Balanced e_i: F1 = 53.871%

10000 Rows, Unbalanced s_n: F1 = 52.25%


10000 Rows, Balanced s_n: F1 = 61.23499999999999%

10000 Rows, Unbalanced f_t: F1 = 70.23899999999999%


10000 Rows, Balanced f_t: F1 = 68.594%

10000 Rows, Unbalanced p_j: F1 = 59.239%


10000 Rows, Balanced p_j: F1 = 57.342000000000006%

## Make plots to indicate the difference between balanced and unbalanced EI data